In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.fs as fs
import pandas as pd
from pathlib import Path

In [2]:
import os
os.getcwd()


'/home/pc/dev/homework/module-3'

In [7]:
import pyarrow.parquet as pq
from pathlib import Path

file = next(Path("./data/green").glob("*.parquet"))

df = pq.read_table(file).to_pandas()

df.head(5)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-12-01 00:29:37,2020-12-01 00:32:51,N,1.0,75,75,1.0,0.59,4.5,0.5,0.5,1.16,0.0,NaN,0.3,6.96,1.0,1.0,0.00
1,2.0,2020-12-01 00:41:46,2020-12-01 00:46:31,N,1.0,75,74,1.0,1.24,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1.0,1.0,0.00
2,2.0,2020-12-01 00:05:46,2020-12-01 00:10:48,N,1.0,244,243,2.0,1.19,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1.0,1.0,0.00
3,2.0,2020-11-30 23:59:17,2020-12-01 00:16:06,N,1.0,75,68,1.0,5.08,17.0,0.5,0.5,2.00,0.0,NaN,0.3,23.05,1.0,1.0,2.75
4,2.0,2020-12-01 00:57:03,2020-12-01 01:00:29,N,1.0,74,263,1.0,1.24,5.5,0.5,0.5,0.00,0.0,NaN,0.3,9.55,2.0,1.0,2.75


In [5]:
LOCAL_DATA_DIR = Path("./data/green")

sample_file = next(LOCAL_DATA_DIR.glob("*.parquet"))
table = pq.read_table(sample_file)

print(table.schema)

VendorID: double
lpep_pickup_datetime: string
lpep_dropoff_datetime: string
store_and_fwd_flag: string
RatecodeID: double
PULocationID: int64
DOLocationID: int64
passenger_count: double
trip_distance: double
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
ehail_fee: double
improvement_surcharge: double
total_amount: double
payment_type: double
trip_type: double
congestion_surcharge: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2786


In [8]:
import pyarrow as pa

TARGET_SCHEMA = pa.schema([
    pa.field("VendorID", pa.int64()),
    pa.field("lpep_pickup_datetime", pa.timestamp("us")),
    pa.field("lpep_dropoff_datetime", pa.timestamp("us")),
    pa.field("store_and_fwd_flag", pa.string()),
    pa.field("RatecodeID", pa.int64()),
    pa.field("PULocationID", pa.int64()),
    pa.field("DOLocationID", pa.int64()),
    pa.field("passenger_count", pa.int64()),
    pa.field("trip_distance", pa.float64()),
    pa.field("fare_amount", pa.float64()),
    pa.field("extra", pa.float64()),
    pa.field("mta_tax", pa.float64()),
    pa.field("tip_amount", pa.float64()),
    pa.field("tolls_amount", pa.float64()),
    pa.field("ehail_fee", pa.float64()),
    pa.field("improvement_surcharge", pa.float64()),
    pa.field("total_amount", pa.float64()),
    pa.field("payment_type", pa.int64()),
    pa.field("trip_type", pa.int64()),
    pa.field("congestion_surcharge", pa.float64()),
])


In [9]:
df = pq.read_table(file).to_pandas()

# datetime 복구
df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])

# float → int (nullable 안전)
int_cols = [
    "VendorID",
    "RatecodeID",
    "passenger_count",
    "payment_type",
    "trip_type"
]

for c in int_cols:
    df[c] = df[c].astype("Int64")  # pandas nullable int


In [10]:
table_fixed = pa.Table.from_pandas(
    df,
    schema=TARGET_SCHEMA,
    preserve_index=False
)


In [11]:
from pathlib import Path
import pyarrow.parquet as pq

OUT_DIR = Path("./data/green_fixed")
OUT_DIR.mkdir(exist_ok=True)

out_file = OUT_DIR / file.name

pq.write_table(
    table_fixed,
    out_file,
    compression="snappy"
)

print(out_file)


data/green_fixed/green_tripdata_2020-12.parquet


In [12]:
pq.read_table(out_file).schema


VendorID: int64
lpep_pickup_datetime: timestamp[us]
lpep_dropoff_datetime: timestamp[us]
store_and_fwd_flag: string
RatecodeID: int64
PULocationID: int64
DOLocationID: int64
passenger_count: int64
trip_distance: double
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
ehail_fee: double
improvement_surcharge: double
total_amount: double
payment_type: int64
trip_type: int64
congestion_surcharge: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2596

In [13]:
import pyarrow.parquet as pq
from pathlib import Path

fixed_file = Path("./data/green_fixed").glob("*.parquet")
fixed_file = next(fixed_file)

df_check = pq.read_table(fixed_file).to_pandas()

df_check.head(5)


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2020-12-01 00:29:37,2020-12-01 00:32:51,N,1,75,75,1,0.59,4.5,0.5,0.5,1.16,0.0,NaN,0.3,6.96,1,1,0.00
1,2,2020-12-01 00:41:46,2020-12-01 00:46:31,N,1,75,74,1,1.24,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1,1,0.00
2,2,2020-12-01 00:05:46,2020-12-01 00:10:48,N,1,244,243,2,1.19,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1,1,0.00
3,2,2020-11-30 23:59:17,2020-12-01 00:16:06,N,1,75,68,1,5.08,17.0,0.5,0.5,2.00,0.0,NaN,0.3,23.05,1,1,2.75
4,2,2020-12-01 00:57:03,2020-12-01 01:00:29,N,1,74,263,1,1.24,5.5,0.5,0.5,0.00,0.0,NaN,0.3,9.55,2,1,2.75


In [15]:
from pathlib import Path

SRC_DIR = Path("./data/green")
OUT_DIR = Path("./data/green_fixed")
OUT_DIR.mkdir(exist_ok=True)


In [16]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

int_cols = [
    "VendorID",
    "RatecodeID",
    "passenger_count",
    "payment_type",
    "trip_type",
]

for file in sorted(SRC_DIR.glob("*.parquet")):
    print(f"Processing: {file.name}")

    # parquet → pandas
    df = pq.read_table(file).to_pandas()

    # datetime 복구
    df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
    df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])

    # float → nullable int
    for c in int_cols:
        df[c] = df[c].astype("Int64")

    # pandas → arrow (스키마 강제)
    table_fixed = pa.Table.from_pandas(
        df,
        schema=TARGET_SCHEMA,
        preserve_index=False
    )

    # 저장
    out_file = OUT_DIR / file.name
    pq.write_table(
        table_fixed,
        out_file,
        compression="snappy"
    )

print("✅ 2019·2020 parquet 전체 변환 완료")


Processing: green_tripdata_2019-01.parquet
Processing: green_tripdata_2019-02.parquet
Processing: green_tripdata_2019-03.parquet
Processing: green_tripdata_2019-04.parquet
Processing: green_tripdata_2019-05.parquet
Processing: green_tripdata_2019-06.parquet
Processing: green_tripdata_2019-07.parquet
Processing: green_tripdata_2019-08.parquet
Processing: green_tripdata_2019-09.parquet
Processing: green_tripdata_2019-10.parquet
Processing: green_tripdata_2019-11.parquet
Processing: green_tripdata_2019-12.parquet
Processing: green_tripdata_2020-01.parquet
Processing: green_tripdata_2020-02.parquet
Processing: green_tripdata_2020-03.parquet
Processing: green_tripdata_2020-04.parquet
Processing: green_tripdata_2020-05.parquet
Processing: green_tripdata_2020-06.parquet
Processing: green_tripdata_2020-07.parquet
Processing: green_tripdata_2020-08.parquet
Processing: green_tripdata_2020-09.parquet
Processing: green_tripdata_2020-10.parquet
Processing: green_tripdata_2020-11.parquet
Processing:

In [17]:
import pyarrow.parquet as pq

test_file = next(OUT_DIR.glob("*.parquet"))

pq.read_table(test_file).schema


VendorID: int64
lpep_pickup_datetime: timestamp[us]
lpep_dropoff_datetime: timestamp[us]
store_and_fwd_flag: string
RatecodeID: int64
PULocationID: int64
DOLocationID: int64
passenger_count: int64
trip_distance: double
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
ehail_fee: double
improvement_surcharge: double
total_amount: double
payment_type: int64
trip_type: int64
congestion_surcharge: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2596

In [18]:
pq.read_table(test_file).to_pandas().head()


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2020-12-01 00:29:37,2020-12-01 00:32:51,N,1,75,75,1,0.59,4.5,0.5,0.5,1.16,0.0,NaN,0.3,6.96,1,1,0.00
1,2,2020-12-01 00:41:46,2020-12-01 00:46:31,N,1,75,74,1,1.24,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1,1,0.00
2,2,2020-12-01 00:05:46,2020-12-01 00:10:48,N,1,244,243,2,1.19,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1,1,0.00
3,2,2020-11-30 23:59:17,2020-12-01 00:16:06,N,1,75,68,1,5.08,17.0,0.5,0.5,2.00,0.0,NaN,0.3,23.05,1,1,2.75
4,2,2020-12-01 00:57:03,2020-12-01 01:00:29,N,1,74,263,1,1.24,5.5,0.5,0.5,0.00,0.0,NaN,0.3,9.55,2,1,2.75
